In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage containing the table
url = "https://www.ultimatetennisstatistics.com/tournamentEvent?tournamentEventId=4626&tab=results"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table by its ID ("resultsTable")
    table = soup.find("table", {"id": "resultsTable"})

    if table:
        # Create an empty DataFrame to store the data
        data = []
        
        # Initialize a variable to keep track of the row index
        row_index = 0
        
        # Extract and append the data from the table to the DataFrame
        for row in table.find_all("tr"):
            player_cell = row.find("td", {"class": "player"})
            score_cells = row.find_all("td", {"class": "score"})

            if player_cell:
                player_name = player_cell.text.strip()
                # Extract nationality from the title attribute of the img tag
                nationality = player_cell.find("img")["title"]

                # Initialize variables for tiebreak sets and games in each set
                tiebreak_set1 = tiebreak_set2 = tiebreak_set3 = 0
                games_set1 = games_set2 = games_set3 = None

                if score_cells:
                    scores = [cell.text.strip() for cell in score_cells]

                    # Extract tiebreak information and update set scores
                    if len(scores) > 0 and "(" in scores[0]:
                        tiebreak_set1 = 1
                        games_set1 = int(scores[0].split("(")[1].split(")")[0])
                        scores[0] = scores[0].split("(")[0].strip()
                    if len(scores) > 1 and "(" in scores[1]:
                        tiebreak_set2 = 1
                        games_set2 = int(scores[1].split("(")[1].split(")")[0])
                        scores[1] = scores[1].split("(")[0].strip()
                    if len(scores) > 2 and "(" in scores[2]:
                        tiebreak_set3 = 1
                        games_set3 = int(scores[2].split("(")[1].split(")")[0])
                        scores[2] = scores[2].split("(")[0].strip()

                # Determine the 'stage' based on row index
                row_index += 1
                if row_index in [3, 4, 11, 12, 19, 20, 27, 28, 35, 36, 43, 44, 51, 52, 59, 60]:
                    stage = "quarterfinal"
                elif row_index in [5, 6, 13, 14, 21, 22, 29, 30]:
                    stage = "semifinal"
                elif row_index in [7, 8, 15, 16]:
                    stage = "final"
                else:
                    stage = "roundrobin"

                data.append([player_name, nationality, scores[0] if len(scores) > 0 else None, scores[1] if len(scores) > 1 else None, scores[2] if len(scores) > 2 else None, tiebreak_set1, tiebreak_set2, tiebreak_set3, games_set1, games_set2, games_set3, stage])

        # Create a Pandas DataFrame from the data
        df = pd.DataFrame(data, columns=["Player", "Nationality", "Set1", "Set2", "Set3", "Tiebreak_Set1", "Tiebreak_Set2", "Tiebreak_Set3", "Games_Set1", "Games_Set2", "Games_Set3", "Stage"])

        # Display the DataFrame
        print(df)

else:
    print("Failed to fetch the webpage. Status code:", response.status_code)


                   Player Nationality Set1 Set2  Set3  Tiebreak_Set1  \
0                 B. Gojo         CRO    6    3     6              0   
1                 B. Gojo         CRO    6    3     6              0   
2                 S. Baez         ARG    1    6     3              0   
3             J. Thompson         AUS    4    7     6              0   
4           T. Griekspoor         NED    6    5     3              0   
5                B. Coric         CRO    6    6  None              0   
6           T. Kokkinakis         AUS    4    3  None              0   
7           D. Shapovalov         CAN    6    6  None              0   
8           T. Kokkinakis         AUS    2    4  None              0   
9                B. Coric         CRO    6    7     4              0   
10               B. Coric         CRO    6    7  None              0   
11           F. Cerundolo         ARG    4    6  None              0   
12           A. De Minaur         AUS    5    6     6           

In [31]:
df

,Player,Nationality,Set1,Set2,Set3,Tiebreak_Set1,Tiebreak_Set2,Tiebreak_Set3,Games_Set1,Games_Set2,Games_Set3,Stage
0,B. Gojo,CRO,6,3,6,0,0,0,NaN,NaN,NaN,roundrobin
1,B. Gojo,CRO,6,3,6,0,0,0,NaN,NaN,NaN,roundrobin
2,S. Baez,ARG,1,6,3,0,0,0,NaN,NaN,NaN,quarterfinal
3,J. Thompson,AUS,4,7,6,0,0,0,NaN,NaN,NaN,quarterfinal
4,T. Griekspoor,NED,6,5,3,0,0,0,NaN,NaN,NaN,semifinal
5,B. Coric,CRO,6,6,None,0,0,0,NaN,NaN,NaN,semifinal
6,T. Kokkinakis,AUS,4,3,None,0,0,0,NaN,NaN,NaN,final
7,D. Shapovalov,CAN,6,6,None,0,0,0,NaN,NaN,NaN,final
8,T. Kokkinakis,AUS,2,4,None,0,0,0,NaN,NaN,NaN,roundrobin
9,B. Coric,CRO,6,7,4,0,1,0,NaN,8.0,NaN,roundrobin


In [41]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage containing the table
url = "https://www.ultimatetennisstatistics.com/tournamentEvent?tournamentEventId=4626&tab=results"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table by its ID ("resultsTable")
    table = soup.find("table", {"id": "resultsTable"})

    # Initialize a list to store the href links
    href_links = []

    if table:
        # Find all rows in the table
        rows = table.find_all("tr")

        # Loop through each row in the table
        for row in rows:
            # Find the specific "player" td element in the row
            player_cell = row.find("td", {"class": "player"})

            # Check if the player_cell is not None
            if player_cell:
                # Find the player's href link in the first column
                player_link = player_cell.find("a")

                # Check if the player_link is not None and get the href attribute
                if player_link:
                    href = player_link.get("href")
                    href_links.append(href)

    # Print the retrieved href links
    for link in href_links:
        print(link)

else:
    print("Failed to fetch the webpage. Status code:", response.status_code)


/playerProfile?playerId=27140
/playerProfile?playerId=27140
/playerProfile?playerId=46888
/playerProfile?playerId=11415
/playerProfile?playerId=34651
/playerProfile?playerId=6418
/playerProfile?playerId=6409
/playerProfile?playerId=33214
/playerProfile?playerId=6409
/playerProfile?playerId=6418
/playerProfile?playerId=6418
/playerProfile?playerId=46887
/playerProfile?playerId=45105
/playerProfile?playerId=22139
/playerProfile?playerId=45105
/playerProfile?playerId=5220
/playerProfile?playerId=44824
/playerProfile?playerId=45105
/playerProfile?playerId=11174
/playerProfile?playerId=11174
/playerProfile?playerId=46888
/playerProfile?playerId=6418
/playerProfile?playerId=5131
/playerProfile?playerId=32067
/playerProfile?playerId=33214
/playerProfile?playerId=44481
/playerProfile?playerId=44481
/playerProfile?playerId=6029
/playerProfile?playerId=5220
/playerProfile?playerId=5793
/playerProfile?playerId=44824
/playerProfile?playerId=52132
/playerProfile?playerId=11174
/playerProfile?player

In [50]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage containing the table
base_url = "https://www.ultimatetennisstatistics.com"
url = "https://www.ultimatetennisstatistics.com/tournamentEvent?tournamentEventId=4626&tab=results"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table by its ID ("resultsTable")
    table = soup.find("table", {"id": "resultsTable"})

    # Initialize a list to store the href links
    href_links = []

    if table:
        # Find all rows in the table
        rows = table.find_all("tr")

        # Loop through each row in the table
        for row in rows:
            # Find the specific "player" td element in the row
            player_cell = row.find("td", {"class": "player"})

            # Check if the player_cell is not None
            if player_cell:
                # Find the player's href link in the first column
                player_link = player_cell.find("a")

                # Check if the player_link is not None and get the href attribute
                if player_link:
                    href = player_link.get("href")
                    href_links.append(href)

    # Access each URL and retrieve information from the three tables
    for href in href_links:
        player_url = base_url + href
        player_response = requests.get(player_url)

        if player_response.status_code == 200:
            player_soup = BeautifulSoup(player_response.text, "html.parser")
            
            # Find the three tables with class "table table-condensed text-nowrap"
            tables = player_soup.find_all("table", {"class": "table table-condensed text-nowrap"})
            
            # Loop through the three tables
            for i, table in enumerate(tables):
                # Print or process information from each table
                print(f"Table {i + 1}:")
                for row in table.find_all("tr"):
                    # Process each row in the table as needed
                    # Example: Print the text content of each row
                    print(row.text.strip())
                print()  # Separate tables with a blank line
            
        else:
            print("Failed to fetch player's webpage. Status code:", player_response.status_code)

else:
    print("Failed to fetch the webpage. Status code:", response.status_code)


''